In [17]:
import pandas as pd
import gmaps
from config1 import g_key
import requests 
import matplotlib.pyplot as plt


In [2]:
#import the csv file
city_data_df=pd.read_csv('weather_Database_challenge.csv')
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current status,Rain inches(last 3 hrs),Snow inches(last 3 hrs)
0,0,puerto ayora,-0.74,-90.35,70.00,87,5,9.69,clear sky,0.0,0.0
1,1,bredasdorp,-34.53,20.04,69.80,68,99,13.87,overcast clouds,0.0,0.0
2,2,tabas,33.60,56.92,59.00,19,0,2.24,clear sky,0.0,0.0
3,3,atuona,-9.80,-139.03,80.64,67,59,9.42,broken clouds,0.0,0.0
4,4,dikson,73.51,80.55,9.37,95,66,7.81,broken clouds,0.0,0.0


In [3]:
gmaps.configure(api_key=g_key)
#inputing the minimum temperature and maximum temperature preference
min_temperature=float(input("what is you prefer lowest temperature"))
max_tempearture=float(input("what is you prefer highest temperature"))
rain_pre=input("Do you like raining and snowing(yes/no only)")
            

what is you prefer lowest temperature75
what is you prefer highest temperature85
Do you like raining and snowing(yes/no only)no


In [4]:
# Filter dataset to find cities that fit the temperature criteria
favorite_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_tempearture) & (city_data_df['Max Temp'] >= min_temperature)]
# Filter dataset to fit raining criteria
if (rain_pre == 'yes'):
    preferred_cities_df = favorite_cities_df.loc[(favorite_cities_df['Rain inches(last 3 hrs)'] > 0) | (favorite_cities_df['Snow inches(last 3 hrs)'] > 0)]
else:
    preferred_cities_df = favorite_cities_df.loc[(favorite_cities_df['Rain inches(last 3 hrs)'] == 0) & (favorite_cities_df['Snow inches(last 3 hrs)'] == 0)]
    
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.head()
    

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current status,Rain inches(last 3 hrs),Snow inches(last 3 hrs)
3,3,atuona,-9.80,-139.03,80.64,67,59,9.42,broken clouds,0.0,0.0
13,13,itoman,26.12,127.67,75.20,57,20,16.11,few clouds,0.0,0.0
21,21,biltine,14.53,20.93,75.16,18,0,15.82,clear sky,0.0,0.0
32,32,kabanjahe,3.10,98.49,84.20,94,75,10.29,broken clouds,0.0,0.0
35,35,nyimba,-14.56,30.81,83.73,44,5,9.78,clear sky,0.0,0.0


In [5]:
hotel_df=preferred_cities_df[['City','Max Temp','Current status','Lat','Lng']].copy()
hotel_df["Hotel Name"]=''
hotel_df.head()

,City,Max Temp,Current status,Lat,Lng,Hotel Name
3,atuona,80.64,broken clouds,-9.80,-139.03,
13,itoman,75.20,few clouds,26.12,127.67,
21,biltine,75.16,clear sky,14.53,20.93,
32,kabanjahe,84.20,broken clouds,3.10,98.49,
35,nyimba,83.73,clear sky,-14.56,30.81,


In [6]:
# Parameters set up
params={"radius":5000,
        "type":"lodging",
        "key":g_key}

In [7]:
#Dataframe Iteration

for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
    params['location']=f"{lat},{lng}"
    google_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels=requests.get(google_url,params=params).json()
    
    
    try:
        hotel_df.loc[index,'Hotel Name']= hotels['results'][0]['name']
    except(IndexError):
        print('Hotel not found')
        pass
    

Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found


In [8]:
hotel_df.head()

,City,Max Temp,Current status,Lat,Lng,Hotel Name
3,atuona,80.64,broken clouds,-9.80,-139.03,Villa Enata
13,itoman,75.20,few clouds,26.12,127.67,Southern Beach Hotel & Resort Okinawa
21,biltine,75.16,clear sky,14.53,20.93,Hôtel Trois Étoiles de Biltine
32,kabanjahe,84.20,broken clouds,3.10,98.49,Pelindung Bawah Losmen
35,nyimba,83.73,clear sky,-14.56,30.81,Dali Dali Lodge


In [12]:

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [13]:
hotel_information=[info_box_template.format(**row) for index,row in hotel_df.iterrows()]

In [14]:
hotel_df

,City,Max Temp,Current status,Lat,Lng,Hotel Name
3,atuona,80.64,broken clouds,-9.80,-139.03,Villa Enata
13,itoman,75.20,few clouds,26.12,127.67,Southern Beach Hotel & Resort Okinawa
21,biltine,75.16,clear sky,14.53,20.93,Hôtel Trois Étoiles de Biltine
32,kabanjahe,84.20,broken clouds,3.10,98.49,Pelindung Bawah Losmen
35,nyimba,83.73,clear sky,-14.56,30.81,Dali Dali Lodge
...,...,...,...,...,...,...
531,bathsheba,80.60,broken clouds,13.22,-59.52,Atlantis Hotel
534,grand gaube,82.40,few clouds,-20.01,57.66,Veranda Paul et Virginie Hotel & Spa
545,auki,76.08,overcast clouds,12.18,6.51,
546,sokoni,78.19,overcast clouds,-12.89,28.78,


In [20]:
locations=hotel_df[["Lat","Lng"]]
cities=hotel_df["City"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_information)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
plt.savefig("WeatherPy_vacation_map.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [21]:
output_data_file="WeatherPy_vacation.csv"
hotel_df.to_csv(output_data_file,index_label="City_ID")